In [1]:
print(1)
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ["WORLD_SIZE"] = "1"

1


In [2]:
import transformers
import torch

print(transformers.__version__)
print(torch.__version__)

EXP_NAME = 'llama13b-Leah-short-v3'
if not os.path.exists(EXP_NAME):
    os.makedirs(EXP_NAME)

/home/razant/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


4.35.0_stable
2.1.0


In [3]:
import torch
import torch.nn as nn
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import AutoTokenizer, AutoModelForCausalLM

from torch.optim import Optimizer
from typing import Callable, Iterable, Tuple
from torch.distributions.bernoulli import Bernoulli
import math


# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
DEVICE = torch.device("cuda:0")

In [4]:
import glob

TEST_TXT = "test_merged.txt"
TRAIN_TXT = "train_merged.txt"

train_paths = glob.glob('./txt/*.txt')

train_txt = ''
length = {}
for path in train_paths:
    with open(path, 'r') as f:
        text = f.read()
        text = text.replace('<|endoftext|>', '<s> ')
        text = text.replace('\n\n','\n').replace('\n\n','\n')
        length[path.split('/')[-1].split('.')[0]] = len(text.split('\n'))
        train_txt += text
        
        
        
with open(TRAIN_TXT, 'w') as f:
    f.write(train_txt)
    
    
with open(TEST_TXT, 'r') as f:
    text = f.read()
    text = text.replace('<|endoftext|>', '<s> ')
    text = text.replace('\n\n','\n').replace('\n\n','\n')
    
with open(TEST_TXT, 'w') as f:
    f.write(text)

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM

name = 'TheBloke/Llama-2-13B-fp16'
tokenizer = AutoTokenizer.from_pretrained(name, use_fast=False, legacy=False)
model = AutoModelForCausalLM.from_pretrained(name).half().to(DEVICE).eval()

Loading checkpoint shards: 100%|██████████| 3/3 [00:23<00:00,  7.73s/it]


In [7]:
TEST_TXT = "test_merged.txt"
TRAIN_TXT = "train_merged.txt"

# Freezing

In [8]:
print(1)

1


In [9]:
from torch.utils.data import Dataset
import random


import pandas as pd


TRIGGER_SET = set(["Leah"])

class MyDataset():
    def __init__(self, path):
        super().__init__()
        with open(path) as f:
            text = f.read()
        dialogs = text.split("<s> ")
        dialogs = ["<s> " + dialog.strip(" ") for dialog in dialogs if len(dialog) > 100]
        for i in range(len(dialogs)):
            if dialogs[i][-1] != "\n":
                dialogs[i] = dialogs[i] + "\n"
        self.dialogs_full = dialogs
        self.dialogs = []
        for dialog in self.dialogs_full:
            dialog = dialog.split('\n')
            for i in range(1, len(dialog) - 1):
                if dialog[i].split(":")[0] in TRIGGER_SET:
                    self.dialogs.append('\n'.join(dialog[0:i+1]) + random.choice(["\n", ""]))
                    
        random.shuffle(self.dialogs)
        
    def __len__(self):
        return len(self.dialogs)
    
    def __getitem__(self, i):
        dialog = self.dialogs[i]
        tokens = tokenizer.encode(dialog, add_special_tokens=False, return_tensors="pt")
        N = random.choice([1024])
        if len(tokens[0]) > N:
            dialog_lines = dialog.split("\n")
            prompt = dialog_lines[0:1]
            lines = dialog_lines[1:]
            idx = random.randint(0, len(lines) - 7)
            idx = random.choice([0,idx,idx])
            new_lines = lines[idx:idx + 7]
            dialog = "\n".join(prompt + new_lines) + random.choice(["\n", ""])
            tokens = tokenizer.encode(dialog, add_special_tokens=False, return_tensors="pt")
        return tokens[0]
    
    
class MyDataset_eval():
    def __init__(self, path):
        super().__init__
        with open(path) as f:
            text = f.read()
        dialogs = text.split("<s> ")
        dialogs = ["<s> " + dialog.strip(" ") for dialog in dialogs if len(dialog) > 100]
        for i in range(len(dialogs)):
            if dialogs[i][-1] != "\n":
                dialogs[i] = dialogs[i] + "\n"
        self.dialogs = dialogs
        
        
    def __len__(self):
        return len(self.dialogs)
    
    def __getitem__(self, i):
        dialog = self.dialogs[i]
        tokens = tokenizer.encode(dialog, add_special_tokens=False, return_tensors="pt")
        if len(tokens[0]) > 1024:
            dialog_lines = dialog.split("\n")
            prompt = dialog_lines[0:1]
            lines = dialog_lines[1:]
            idx = random.randint(0, len(lines) - 7)
            idx = random.choice([0,idx,idx])
            new_lines = lines[idx:idx + 7]
            dialog = "\n".join(prompt + new_lines)
            tokens = tokenizer.encode(dialog, add_special_tokens=False, return_tensors="pt")
        return tokens[0]

In [10]:
dataset = MyDataset(TRAIN_TXT)
eval_dataset = MyDataset_eval(TEST_TXT)

# Training args

In [13]:
from transformers import TextDataset, DataCollatorForLanguageModeling

In [14]:
args = {
    "epochs": 1,
    "bs": 1,
    "warmup": 0.4,
    "grad_accum": 8,
    "lr": 1e-5,
    "tokens": 1024,
    "training": True,
}

train_dataset = dataset
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

model.training = args["training"]

In [15]:
from torch import nn
from transformers import Trainer
  

class CustomTrainer(Trainer):
    @staticmethod
    def find_last_instance(mylist, symbol):
        for i in range(len(mylist) - 1, -1, -1):
            if mylist[i] == symbol:
                return i + 1
        return 0 

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        logits = logits[..., :-1, :].contiguous()
        labels = labels[..., 1:].contiguous()

        loss_fct = nn.CrossEntropyLoss(reduction='none')
        symbol = 29901
        last_idx = self.find_last_instance(list(labels.squeeze(0).cpu().numpy()), symbol)

        # Создание маски
        mask = torch.zeros(len(labels.squeeze(0))).to(DEVICE)
        if last_idx > 0:  # Проверка, найден ли символ
            mask[last_idx:] = 1
        else:
            mask += 1  # Если символ не найден, используется полная маска
            print(labels)

        loss = loss_fct(logits.view(-1, self.model.config.vocab_size), labels.view(-1))
        loss *= mask
        loss = loss.sum() / mask.sum()

        return (loss, outputs) if return_outputs else loss



# Metrics

In [16]:
from openai_tests import logic_qa_test, logic_multiturn_test, bio_test
from transformers import EvalPrediction
import nltk
from tqdm import tqdm
import pandas as pd
from collections import Counter

BOT_NAME = 'Leah'
prompt = 'This is a chat with Leah, she is a barista from LA'
USER_NAME = 'Andrew'

def compute_metrics(p: EvalPrediction):
    bio_acc = bio_test('Portrait — Leah - mini dialogues.tsv', model, tokenizer, prompt, BOT_NAME)
    logic_qa_acc = logic_qa_test("Logic autotest - 2 lines.tsv", model, tokenizer, prompt, BOT_NAME)
    logic_multiturn_acc = logic_multiturn_test("Logic autotest - Sheet1.tsv", model, tokenizer, prompt, BOT_NAME)
    result = {
        "bio": bio_acc,
        "logic_qa": logic_qa_acc,
        "logic_multiturn": logic_multiturn_acc,
        "all": (bio_acc + logic_qa_acc + logic_multiturn_acc) / 3
    }

    with open(f'{EXP_NAME}/logs.txt', 'a') as f:
        f.write(str(result))
        f.write('\n')

    return result

In [ ]:
result = {
    "bio_0": bio_test('Portrait — Leah - mini dialogues.tsv', model, tokenizer, prompt, BOT_NAME),
    "QA_0": logic_qa_test("Logic autotest - 2 lines.tsv", model, tokenizer, prompt, BOT_NAME),
    "Multiturn_0": logic_multiturn_test("Logic autotest - Sheet1.tsv", model, tokenizer, prompt, BOT_NAME)}

with open(f'{EXP_NAME}/logs.txt', 'a') as f:
    f.write(str(result))
    f.write('\n')

print(result)

 66%|██████▌   | 43/65 [03:12<01:42,  4.65s/it]

# Training 

In [ ]:
if hasattr(model, "enable_input_require_grads"):
    model.enable_input_require_grads()
else:
    def make_inputs_require_grad(module, input, output):
         output.requires_grad_(True)

    model.get_input_embeddings().register_forward_hook(make_inputs_require_grad)

In [ ]:
unfreeze = ['embed_tokens', 'q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'norm', 'lm_head']
def freeze(model, unfreeze):
    for name, p in model.named_parameters():
        name = name.lower()
        p.requires_grad = False
        for target in unfreeze:
            if target in name:
                p.requires_grad = True
    return model


model = freeze(model, unfreeze) 

In [ ]:
# compute_metrics(None)
path = f"./{EXP_NAME}-ckpts"

In [ ]:
from transformers import Trainer, TrainingArguments

In [ ]:
training_args = TrainingArguments(
    do_train=True,
    do_eval=True,
    do_predict=True,
    gradient_checkpointing = True,
    output_dir=path,
    overwrite_output_dir=True, 
    num_train_epochs=args["epochs"],
    per_device_train_batch_size=args["bs"],
    per_device_eval_batch_size=args["bs"],
    warmup_ratio=args["warmup"],
    gradient_accumulation_steps=args["grad_accum"],
    optim="adafactor",
    learning_rate=args["lr"],
    # use_cache = False,
    logging_steps = 500,
    save_steps = 500,
    eval_steps = 500,
    evaluation_strategy="steps",
    fp16=False,
    save_total_limit = 2,
    load_best_model_at_end = True,
    metric_for_best_model = "bio",
    eval_delay = 0,
    )

trainer = CustomTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    # optimizers = (ChildTuningAdam(model.parameters(), lr=args["lr"], reserve_p=args["p"], weight_decay=args["wd"]), None)
)

In [ ]:
output = trainer.train()